### Extract data from pdf


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
from llm_synthesis.transformers.pdf_extraction.mistral_pdf_extractor import (
    MistralPDFExtractor,
)

pdf_path = "<path-to-pdf>"  # Replace with your PDF file path
pdf_bytes = open(pdf_path, "rb").read()
extracted_text = MistralPDFExtractor().forward(pdf_bytes)

print(extracted_text)

### Extract figures from extracted data


In [ ]:
from llm_synthesis.transformers.figure_extraction.regex_figure_extractor import (
    FigureExtractorMarkdown,
)

figure_extractor = FigureExtractorMarkdown()

figures = figure_extractor.forward(extracted_text)

In [ ]:
import base64

from IPython.display import Image

# Convert base64 string to image and display
Image(base64.b64decode(figures[0].base64_data))

In [ ]:
from llm_synthesis.models.figure import FigureInfoWithPaper

figure = FigureInfoWithPaper(
    **figures[0].__dict__,
    paper_text=extracted_text,
    si_text="",
)

### Call Claude API for extraction


In [ ]:
from llm_synthesis.transformers.plot_extraction.claude_extraction.plot_data_extraction import (
    ClaudeLinePlotDataExtractor,
)

# Initialize the extractor
extractor = ClaudeLinePlotDataExtractor(model_name="claude-sonnet-4-20250514")

# Perform the extraction
extracted_data = extractor.forward(figure)
print("Extracted data:")
print(extracted_data)

# Track the cost of this operation
cost_info = extractor.get_cost()
cost_info

### Visualize extracted data series with their labels and axis for the chart


In [ ]:
from llm_synthesis.utils.visualization import visulize_line_chart

visulize_line_chart(extracted_data)